In [16]:
import pandas as pd
from datetime import datetime, timedelta
import pytz

# Load user transaction data
data = pd.read_csv('user1.csv')

# Convert 'postDate' column to datetime
data['postDate'] = pd.to_datetime(data['postDate'])

# Filter transactions with 'debit' as 'direction' and 'payment' as class
expenses = data[(data['amount'] < 0) & (data['class'] == 'payment')]

# Define the date range for different time periods - user should be able to select these from a dropdown in the UI
today = datetime.now(pytz.utc)
one_week_ago = today - timedelta(weeks=1)
one_month_ago = today - timedelta(weeks=4)
three_months_ago = today - timedelta(weeks=12)
six_months_ago = today - timedelta(weeks=24)

# Create DataFrames for each time period
time_periods = {
    '1 Week': one_week_ago,
    '1 Month': one_month_ago,
    '3 Months': three_months_ago,
    '6 Months': six_months_ago
}

# Create and store DataFrames for each time period
time_period_dataframes = {}
for period, start_date in time_periods.items():
    filtered_data = expenses[expenses['postDate'] >= start_date]
    time_period_dataframes[period] = filtered_data.groupby('subClass_title')['amount'].sum().reset_index()

# Calculate expenses by 'subClass_title'
subclass_title_expenses = expenses.groupby('subClass_title')['amount'].sum().reset_index()

# Rename columns to 'Category' and 'Amount Spent'
subclass_title_expenses.rename(columns={'subClass_title': 'Category', 'amount': 'Spent'}, inplace=True)

# Make the 'Spent' column positive for display purposes
subclass_title_expenses['Spent'] = subclass_title_expenses['Spent'].abs()

# Add AUD currency symbol to 'Amount Spent' column during printing
subclass_title_expenses['Spent'] = 'AUD ' + subclass_title_expenses['Spent'].astype(str)

# Print the results for both 'subClass_title' and different time periods
print("Expenses by SubClass:")
print(subclass_title_expenses)
print("\nExpenses by Time Period:")
for period, sub_class_data in time_period_dataframes.items():
    print(f"{period}:")

    # Make the 'Amount Spent' column positive for display purposes
    sub_class_data['amount'] = sub_class_data['amount'].abs()

    print(sub_class_data)
    print("\n")

Expenses by SubClass:
                                            Category          Spent
0          Auxiliary Finance and Investment Services  AUD 133376.25
1  Civic, Professional and Other Interest Group S...  AUD 274595.83
2                           Electricity Distribution    AUD 5813.11
3                      Legal and Accounting Services    AUD 15500.0
4          Other Machinery and Equipment Wholesaling      AUD 36.35
5                                Regulatory Services     AUD 1093.7
6                                   School Education   AUD 47736.59

Expenses by Time Period:
1 Week:
Empty DataFrame
Columns: [subClass_title, amount]
Index: []


1 Month:
                  subClass_title  amount
0  Legal and Accounting Services   500.0


3 Months:
                                      subClass_title    amount
0          Auxiliary Finance and Investment Services   7667.50
1  Civic, Professional and Other Interest Group S...  18849.50
2                           Electricity Distri